# Libraries

In [1]:
import json
import pandas as pd 
import numpy as np 
import shutil
import gzip
import plotly.graph_objects as go
from collections import Counter
from sklearn.preprocessing import LabelEncoder

#data preprocessing file
import data_preprocessing

In [2]:
pd.set_option("display.max_rows", None, "display.max_columns", None)

# Load places dataset
places.clean.json.gz file contains information about the places, operational timing and gps location. For this study on recommender system, we extract all the food places in gps location of 53 W and and 141 W & latitude range from 42 N and 83 N. This covers food places in southern cities of Canada and northern parts in the United States of America

In [3]:
data_places = []
with gzip.open('places.clean.json.gz') as f:
    for l in f:
        data_places.append(eval(l))

In [4]:
places = pd.DataFrame(data_places)


In [5]:
#remove places where gps location is not given
places = places[places['gps'].notna()]

#make latitude and longitude columns
places['lat'] = places['gps'].apply(lambda x: x[0])
places['long'] = places['gps'].apply(lambda x: x[1])

In [6]:
canada_us = places[(places['lat']>=42) & (places['lat']<=83) & (places['long']>=-141) & (places['long']<=-53)]
canada_us.set_index('gPlusPlaceId', inplace = True)

In [7]:
canada_us.shape

(313417, 9)

In [8]:
restraunts = pd.read_pickle('../Datasets/food_review.pickle')
restraunts.head(5)

,rating,reviewerName,reviewText,categories,gPlusPlaceId,unixReviewTime,reviewTime,gPlusUserId
0,4.0,william spindler,best war wanton soup red bluff,"[asian restaurant, chinese restaurant]",106591714648856494903,2014-03-13 05:41:36,2014-03-12,100000032416892623125
1,5.0,william spindler,long time favorit mexican food alway consist g...,[mexican restaurant],115827996910815192564,2014-03-13 05:45:13,2014-03-12,100000032416892623125
2,5.0,william spindler,best hot sour soup anywher,[chinese restaurant],116585428624152564242,2014-03-15 05:42:39,2014-03-14,100000032416892623125
3,2.0,richard yocom,open 24 hour plu realli slow sat drive thru 30...,[restaurant],100073820849130920147,2013-08-11 12:15:30,2013-08-10,100000053212755369563
4,1.0,richard yocom,close,"[pizza restaurant, european restaurant]",102471437282277965376,2013-08-17 00:24:50,2013-08-16,100000053212755369563


In [9]:
data_to_model = pd.merge(restraunts,canada_us,left_on = 'gPlusPlaceId',right_on = 'gPlusPlaceId',how = 'inner')

In [10]:
data_to_model.set_index('gPlusPlaceId',inplace = True,drop = True)

In [11]:
data_to_model.head(10)

,rating,reviewerName,reviewText,categories,unixReviewTime,reviewTime,gPlusUserId,name,price,address,hours,phone,closed,gps,lat,long
gPlusPlaceId,,,,,,,,,,,,,,,,
112181651134020562716,5.0,jason wagner,stop dinner mahi mahi special fantast servic g...,"[european restaurant, italian restaurant]",2012-10-18 21:56:05,2012-10-18,100000196778399872657,Pasquales Trattoria,None,"[50 Lakeshore Rd W, Oakville, ON L6Y 2K3, Canada]","[[Monday, [['Closed']]], [Tuesday, [['11:30 am...",(905) 338-9090,False,"[43.440331, -79.672851]",43.440331,-79.672851
109391653021436219474,5.0,heather goldsworthy,best coffe collingwood,[cafe],2013-09-12 18:38:15,2013-09-12,100000427343266187570,Espresso Post,None,"[139 Hurontario St, Collingwood, ON L9Y, Canada]",None,(705) 446-1740,False,"[44.499252, -80.216587]",44.499252,-80.216587
109391653021436219474,5.0,nicholas clayton,locat old post offic outlet histor pedigre bes...,[cafe],2013-05-04 01:35:12,2013-05-03,104975196938413074237,Espresso Post,None,"[139 Hurontario St, Collingwood, ON L9Y, Canada]",None,(705) 446-1740,False,"[44.499252, -80.216587]",44.499252,-80.216587
110864889964667973890,5.0,chris johnson,get right everi time love guy,"[delivery restaurant, event planner, business ...",2011-09-15 21:54:03,2011-09-15,100000524810171549476,"Delivered Dish | Portland, OR",$$,"[Portland, OR]","[[Monday, [['8:00 am--9:00 pm']]], [Tuesday, [...",(503) 239-0100,False,"[45.470342, -122.747085]",45.470342,-122.747085
110864889964667973890,1.0,allison lea,use servic order even show 1 hour 20 minut pro...,"[delivery restaurant, event planner, business ...",2013-11-09 10:28:01,2013-11-08,100537426867747151661,"Delivered Dish | Portland, OR",$$,"[Portland, OR]","[[Monday, [['8:00 am--9:00 pm']]], [Tuesday, [...",(503) 239-0100,False,"[45.470342, -122.747085]",45.470342,-122.747085
110864889964667973890,1.0,derek espinoza,late half hour like person call 25 minut food ...,"[delivery restaurant, event planner, business ...",2013-09-30 09:25:54,2013-09-29,105608513775015690240,"Delivered Dish | Portland, OR",$$,"[Portland, OR]","[[Monday, [['8:00 am--9:00 pm']]], [Tuesday, [...",(503) 239-0100,False,"[45.470342, -122.747085]",45.470342,-122.747085
106925262113173878216,5.0,jon bredensteiner (willy),best bar area,"[bar & grill, irish pub, cocktail bar]",2014-01-31 11:59:03,2014-01-30,100000566806487162811,O'finnigans Pub,None,"[13601 Washington 99, Everett, WA 98204]","[[Monday, [['11:00 am--2:00 am']]], [Tuesday, ...",(425) 745-4805,False,"[47.874683, -122.27189]",47.874683,-122.271890
106925262113173878216,2.0,david poulton,would never go circumst horribl staff lack kno...,"[bar & grill, irish pub, cocktail bar]",2013-02-12 11:24:05,2013-02-11,104636730914402265439,O'finnigans Pub,None,"[13601 Washington 99, Everett, WA 98204]","[[Monday, [['11:00 am--2:00 am']]], [Tuesday, ...",(425) 745-4805,False,"[47.874683, -122.27189]",47.874683,-122.271890
109723908842622284016,3.0,suzanne lander,love hole in the wal slightli greasi delici de...,"[asian restaurant, chinese restaurant]",2012-11-11 00:03:33,2012-11-10,100000820235275835897,Duk Li Dim Sum Restaurant,$$,"[664 S Weller St, Seattle, WA 98104]","[[Monday, [['7:00 am--7:00 pm']]], [Tuesday, [...",(206) 340-6122,False,"[47.597602, -122.324369]",47.597602,-122.324369


In [12]:
print("Number of users in dataset: ", len(data_to_model['gPlusUserId'].unique()))
print("Number of places in dataset : ", len(data_to_model.index.unique()))

Number of users in dataset:  41568
Number of places in dataset :  43018


In [13]:
data_to_model.to_pickle('data_to_model_temp.pickle')
data_to_model = pd.read_pickle('data_to_model_temp.pickle')

In [14]:
series = data_to_model[['hours']]
series.dropna(inplace = True)
index_ = series.index

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [15]:
le = LabelEncoder()
le.fit(series.index.unique())
series['id'] = le.transform(series.index)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [16]:
series1 = series.drop_duplicates(subset = 'id')

In [17]:
series1 = series1.loc[:,'hours']
series1.head(10)

gPlusPlaceId
112181651134020562716    [[Monday, [['Closed']]], [Tuesday, [['11:30 am...
110864889964667973890    [[Monday, [['8:00 am--9:00 pm']]], [Tuesday, [...
106925262113173878216    [[Monday, [['11:00 am--2:00 am']]], [Tuesday, ...
109723908842622284016    [[Monday, [['7:00 am--7:00 pm']]], [Tuesday, [...
118279746793420842563    [[Monday, [['10:00 am--12:00 am']]], [Tuesday,...
117375003782548234778    [[Monday, [['6:00 am--12:00 am']]], [Tuesday, ...
108777548128881330137    [[Monday, [['8:00 am--8:00 pm']]], [Tuesday, [...
107377745432053659364    [[Monday, [['11:00 am--10:00 pm']]], [Tuesday,...
108726835119576698559    [[Monday, [['11:00 am--11:00 pm']]], [Tuesday,...
100691533337424531663    [[Monday, [['11:30 am--2:00 am']], 1], [Tuesda...
Name: hours, dtype: object

In [31]:
availability = data_preprocessing.available_times(series1)


In [34]:
availability = availability.apply(pd.Series)
availability.fillna(0,inplace = True)
availability.head(10)

,open_days,Dinner,Lunch,Breakfast
gPlusPlaceId,,,,
112181651134020562716,6.0,1.0,0.0,0.0
110864889964667973890,7.0,1.0,0.0,0.0
106925262113173878216,7.0,1.0,1.0,0.0
109723908842622284016,7.0,1.0,1.0,1.0
118279746793420842563,7.0,1.0,1.0,0.0
117375003782548234778,7.0,1.0,1.0,1.0
108777548128881330137,7.0,1.0,0.0,1.0
107377745432053659364,7.0,1.0,1.0,0.0
108726835119576698559,7.0,1.0,1.0,0.0


In [35]:
availability.shape

(33098, 4)

In [36]:
#merge with canada_us dataframe
data_to_model_final = pd.merge(data_to_model,availability, left_on = data_to_model.index,right_on = availability.index, how = 'inner')

In [37]:
data_to_model_final.head(5)

,key_0,rating,reviewerName,reviewText,categories,unixReviewTime,reviewTime,gPlusUserId,name,price,address,hours,phone,closed,gps,lat,long,open_days,Dinner,Lunch,Breakfast
0,112181651134020562716,5.0,jason wagner,stop dinner mahi mahi special fantast servic g...,"[european restaurant, italian restaurant]",2012-10-18 21:56:05,2012-10-18,100000196778399872657,Pasquales Trattoria,None,"[50 Lakeshore Rd W, Oakville, ON L6Y 2K3, Canada]","[[Monday, [['Closed']]], [Tuesday, [['11:30 am...",(905) 338-9090,False,"[43.440331, -79.672851]",43.440331,-79.672851,6.0,1.0,0.0,0.0
1,110864889964667973890,5.0,chris johnson,get right everi time love guy,"[delivery restaurant, event planner, business ...",2011-09-15 21:54:03,2011-09-15,100000524810171549476,"Delivered Dish | Portland, OR",$$,"[Portland, OR]","[[Monday, [['8:00 am--9:00 pm']]], [Tuesday, [...",(503) 239-0100,False,"[45.470342, -122.747085]",45.470342,-122.747085,7.0,1.0,0.0,0.0
2,110864889964667973890,1.0,allison lea,use servic order even show 1 hour 20 minut pro...,"[delivery restaurant, event planner, business ...",2013-11-09 10:28:01,2013-11-08,100537426867747151661,"Delivered Dish | Portland, OR",$$,"[Portland, OR]","[[Monday, [['8:00 am--9:00 pm']]], [Tuesday, [...",(503) 239-0100,False,"[45.470342, -122.747085]",45.470342,-122.747085,7.0,1.0,0.0,0.0
3,110864889964667973890,1.0,derek espinoza,late half hour like person call 25 minut food ...,"[delivery restaurant, event planner, business ...",2013-09-30 09:25:54,2013-09-29,105608513775015690240,"Delivered Dish | Portland, OR",$$,"[Portland, OR]","[[Monday, [['8:00 am--9:00 pm']]], [Tuesday, [...",(503) 239-0100,False,"[45.470342, -122.747085]",45.470342,-122.747085,7.0,1.0,0.0,0.0
4,106925262113173878216,5.0,jon bredensteiner (willy),best bar area,"[bar & grill, irish pub, cocktail bar]",2014-01-31 11:59:03,2014-01-30,100000566806487162811,O'finnigans Pub,None,"[13601 Washington 99, Everett, WA 98204]","[[Monday, [['11:00 am--2:00 am']]], [Tuesday, ...",(425) 745-4805,False,"[47.874683, -122.27189]",47.874683,-122.271890,7.0,1.0,1.0,0.0


In [42]:
data_to_model_final.columns = ['gPlusPlaceId','rating', 'reviewerName', 'reviewText', 'categories',
       'unixReviewTime', 'reviewTime', 'gPlusUserId', 'name', 'price',
       'address', 'hours', 'phone', 'closed', 'gps', 'lat', 'long',
       'open_days', 'Dinner', 'Lunch', 'Breakfast']

In [43]:
data_to_model_final.shape

(57254, 21)

In [44]:
#save data_to_model as a pickle file
data_to_model_final.to_pickle("../Datasets/food_review_places.pickle")